# Setup

In [94]:
# Import necessary packages
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, TimeoutException
import pandas as pd
import time
from datetime import datetime
import math

# Google review url(s)
url = "https://www.google.com/maps/place/Nespresso/@49.2258269,-123.0014231,17z/data=!4m13!1m7!3m6!1s0x5486765957fb6f03:0x6b717a7e2475f0e8!2sNespresso!8m2!3d49.2258269!4d-122.9992344!10e1!3m4!1s0x5486765957fb6f03:0x6b717a7e2475f0e8!8m2!3d49.2258269!4d-122.9992344";

url_review = "https://www.google.com/maps/place/Nespresso/@49.2258269,-123.0014231,17z/data=!3m1!5s0x54867659512e6deb:0x8a75309114df0b12!4m15!1m7!3m6!1s0x5486765957fb6f03:0x6b717a7e2475f0e8!2sNespresso!8m2!3d49.2258269!4d-122.9992344!10e1!3m6!1s0x5486765957fb6f03:0x6b717a7e2475f0e8!8m2!3d49.2258269!4d-122.9992344!9m1!1b1";

# Run Web Scraper

In [54]:
# Instantiate web driver
driver = webdriver.Chrome("chromedriver");
driver.set_window_size(1024, 600);
driver.maximize_window();
time.sleep(5);

# Go to Google Reviews URL 
driver.get(url);
time.sleep(3);

In [55]:
# Get the total number of Google reviews and overall Google review rating
def get_generalMetrics(xpath):
    driver.find_element("xpath", xpath).location_once_scrolled_into_view;
    time.sleep(1);
    metrics = driver.find_element("xpath", xpath).text.split("\n");
    data = [[metrics[0], metrics[1].split(" ")[0]]];
    df_generalMetrics = pd.DataFrame(data, columns=['Overall Rating','Total Number of Reviews']);
    return df_generalMetrics;

df_generalMetrics = get_generalMetrics('//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[32]/div/div[2]');
df_generalMetrics

,Overall Rating,Total Number of Reviews
0,3.9,328


In [56]:
# Distribution of stars given
def get_reviewStarDistribution(parent_xpath):
    # driver.find_element("xpath", parent_xpath).location_once_scrolled_into_view;
    data = [];
    for i in range(1,6):
        label = driver.find_element("xpath", parent_xpath+'tr['+str(i)+']').get_attribute("aria-label").split(", ");
        time.sleep(1);
        data.append([label[0].split(" ")[0], label[1].split(" ")[0]]);
    df_reviewStar = pd.DataFrame(data, columns=['Stars Given','Number of Reviews']);
    return df_reviewStar;

df_reviewStar = get_reviewStarDistribution('//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[32]/div/div[1]/table/tbody/');
df_reviewStar

,Stars Given,Number of Reviews
0,5,181
1,4,53
2,3,21
3,2,15
4,1,58


In [57]:
# Access full list of Google reviews
driver.get(url_review);
time.sleep(3);

# Sort reviews by "Newest"
driver.find_element("xpath", '//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]/div[7]/div[2]/button').click();

time.sleep(5);

driver.find_element("xpath", '//*[@id="action-menu"]/div[2]').click();

time.sleep(5);

In [58]:
# Scroll to the bottom of the page to get all reviews
def perform_scroll(scrollbar_xpath, y_pixel, number_limit):
    # Get scroll height
    last_height = driver.execute_script("return document.body.scrollHeight");
    time.sleep(5);
    number = 0;

    while True:
        number += 1;

        # Scroll down to bottom
        ele = driver.find_element("xpath", scrollbar_xpath);
        driver.execute_script(f'arguments[0].scrollBy(0, {y_pixel});', ele);

        # Wait to load page

        time.sleep(5);

        # Calculate new scroll height and compare with last scroll height
        print(f'Last Height: {last_height}');

        ele = driver.find_element("xpath", scrollbar_xpath);
        new_height = driver.execute_script("return arguments[0].scrollHeight", ele);

        print(f'New Height: {new_height}');

        if number == number_limit:
            break;

        if new_height == last_height:
            break;
        
        print("continue");
        last_height = new_height;

perform_scroll(
    scrollbar_xpath='//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]', 
    y_pixel=50000,
    number_limit=50
)

Last Height: 604
New Height: 9909
continue
Last Height: 9909
New Height: 15042
continue
Last Height: 15042
New Height: 18830
continue
Last Height: 18830
New Height: 23835
continue
Last Height: 23835
New Height: 27275
continue
Last Height: 27275
New Height: 31040
continue
Last Height: 31040
New Height: 33413
continue
Last Height: 33413
New Height: 35778
continue
Last Height: 35778
New Height: 37770
continue
Last Height: 37770
New Height: 40049
continue
Last Height: 40049
New Height: 42561
continue
Last Height: 42561
New Height: 44517
continue
Last Height: 44517
New Height: 46630
continue
Last Height: 46630
New Height: 48290
continue
Last Height: 48290
New Height: 50371
continue
Last Height: 50371
New Height: 52010
continue
Last Height: 52010
New Height: 53879
continue
Last Height: 53879
New Height: 56221
continue
Last Height: 56221
New Height: 58258
continue
Last Height: 58258
New Height: 60583
continue
Last Height: 60583
New Height: 62700
continue
Last Height: 62700
New Height: 65340
c

In [99]:
# Get the names of all reviewers
def get_namesOfReviewers(div_limit, reviews_xpath):
    i = 1;
    reviewers = [];
    while i <= div_limit:
        try:
            label = driver.find_element(By.XPATH, f'{reviews_xpath}/div[{str(i)}]').get_attribute("aria-label");
            if label != None:
                reviewers.append(label);
            i += 1;
        except NoSuchElementException:
            break;
    return reviewers;

reviewers = get_namesOfReviewers(
    div_limit = 1000, 
    reviews_xpath = '//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]/div[9]'
)

print(reviewers[len(reviewers)-1]);
print(len(reviewers));

George Gunay
328


In [100]:
# Click the 'More' button where the reviews are very long
button_list = driver.find_elements(By.TAG_NAME, 'button');
for button in button_list:
    if button.text == 'More':
        button.click();
        time.sleep(5);

In [101]:
driver.find_element(By.XPATH, '//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]/div[9]').text.split("\n")

['LASHLOVE33',
 '3 reviews',
 'a week ago',
 'NEW',
 'Hit and miss. Staff are either extremely rude or extremely nice.',
 'Like',
 'Share',
 'Response from the owner a week ago',
 'Thank you for taking the time to share your impressions with us. We apologize for the situation. We always aim to offer the best customer experience, and regret learning that you didn’t enjoy your recent visit to our Boutique. Your feedback will be reviewed by our management team to ensure that a consistent quality of service can be offered during subsequent visits. We hope to be able to turn this around the next time you are passing by! ☕',
 'Linda Wilke',
 '4 reviews',
 '3 weeks ago',
 'NEW',
 "Tried calling this boutique to ask some questions about their machines and coffees, but wasn't able to connect directly with them by phone.  We took the risk and drove out from the Fraser Valley (50ish minutes).  We spent less than 5 minutes in the store, as the employee couldn't answer our questions about the diffe

In [105]:
def get_reviewData(reviewers, reviews_xpath):
    data = [];
    for reviewer in reviewers:
        for item in driver.find_element(By.XPATH, reviews_xpath).text.split("\n"):
            row = [];
            if item == reviewer:
                row.append(item);
            else:
                break;
            print(row);
            data.append(row);
        
    return data;

get_reviewData(reviewers=reviewers, reviews_xpath='//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]/div[9]')

['LASHLOVE33']


KeyboardInterrupt: 

# Shutdown Web Scraper

In [106]:
driver.close()